### Importing and Loading the Data from pickle files

In [1]:
#General Imports
import os 
import sys
from dotenv import load_dotenv #importing env file
import pandas as pd
import random
import pickle



In [2]:
import sys 
sys.path.append("..")

from src.advanced_tester import Tester

In [ ]:
#ML Imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

In [ ]:
#NLP
import gensim
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

In [2]:
train_path = '/Users/vishesh/projects/Pricer/data/balanced/train.pkl'
test_path = '/Users/vishesh/projects/Pricer/data/balanced/test.pkl'
with open(train_path,'rb') as f:
    train = pickle.load(f)
with open(test_path,'rb') as f:
    test = pickle.load(f)

In [ ]:
train.

,title,price,category,test_prompt,prompt
0,NaN,1.0,NaN,NaN,NaN
1,NaN,1.0,NaN,NaN,NaN
2,NaN,1.0,NaN,NaN,NaN
3,NaN,1.0,NaN,NaN,NaN
4,NaN,1.0,NaN,NaN,NaN
...,...,...,...,...,...
319995,NaN,1.0,NaN,NaN,NaN
319996,NaN,1.0,NaN,NaN,NaN
319997,NaN,1.0,NaN,NaN,NaN
319998,NaN,1.0,NaN,NaN,NaN


In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        8000 non-null   object 
 1   price        8000 non-null   float64
 2   category     8000 non-null   object 
 3   test_prompt  8000 non-null   object 
 4   prompt       8000 non-null   object 
dtypes: float64(1), object(4)
memory usage: 312.6+ KB


In [ ]:
print(train['title'].iloc[2344])

In [ ]:

print(train['prompt'].iloc[2344])
print(test['test_prompt'].iloc[2344])

### TESTER CLASS

In [ ]:
import math 

### Basic testing

In [ ]:
random.seed(42)

# 2. Define any predictor function
def random_pricer(item):
    return random.randrange(1, 1000)

# 3. Test any function
Tester.test(random_pricer,test)



In [ ]:
#guessing average 
average = train['price'].mean()

def average_pricer(item):
    return average
Tester.test(average_pricer,test)

In [ ]:
train.keys()

In [ ]:
'''
Training is missing the raw details so , 
I processed the data again this time including the raw details 
stored in data Raw
'''

In [ ]:
train_details_path = '/Users/vishesh/projects/Pricer/data/raw/train_details.pkl'
test_details_path = '/Users/vishesh/projects/Pricer/data/raw/test_details.pkl'
with open(train_details_path,'rb') as f:
    train_details = pickle.load(f)
with open(test_details_path , 'rb') as f :
    test_details = pickle.load(f)

In [ ]:
test_details[234].details

### Feature Engineering

In [ ]:
#converting string to dictionary for extracting data/features easily using json 
#in a new features field populated with json from details dict
import json
for detail in train_details:
    detail.features = json.loads(detail.details)
for detail in test_details:
    detail.features = json.loads(detail.details)

train_details[0].features

In [ ]:
#getting the most common features 
from collections import Counter
feature_count = Counter()
for item in train_details:
    for f in item.features.keys():
        feature_count[f]+=1
feature_count.most_common(20)

In [ ]:
#I believe Item Weight Manufacturer brand and Sellers Rank can be useful 
#diving into each categories for better knowledge 

In [ ]:
brands = Counter() 
for item in train_details:
    brand = item.features.get('Brand') 
    if brand :
        brands[brand] +=1
brands.most_common(40)

In [ ]:
#looking the brands it seems like top brands are related to electronics 
TOP_ELECTRONICS_BRANDS = ['hp','dell','lenovo','samsung','asus','sony','canon','apple','intel']

PREMIUM_ELECTRONICS_BRANDS = ['apple', 'sony', 'canon', 'samsung']

TOP_AUTO_PARTS_BRANDS = ['power stop', 'detroit axle', 'dorman', 'buyautoparts!', 'acdelco', 
                         'evan fischer', 'callahan brake parts', 'r1 concepts', 'rareelectrical',
                         'garage-pro', 'spectra premium', 'auto dynasty', 'cardone', 'aps', 
                         'gm', 'walker', 'ebc brakes', 'akkon', 'spec-d tuning', 'tyc', 'a-premium']

TOP_AUTO_ACCESSORIES_BRANDS = ['curt', 'coverking', 'weathertech', 'covercraft', 'k&n']

def is_top_electronics_brand(item):
    brand = item.features.get("Brand")
    return brand and brand.lower() in TOP_ELECTRONICS_BRANDS

def is_premium_electronics_brand(item):
    brand = item.features.get("Brand")
    return brand and brand.lower() in PREMIUM_ELECTRONICS_BRANDS

def is_top_auto_parts_brand(item):
    brand = item.features.get("Brand")
    return brand and brand.lower() in TOP_AUTO_PARTS_BRANDS

def is_top_auto_accessories_brand(item):
    brand = item.features.get("Brand")
    return brand and brand.lower() in TOP_AUTO_ACCESSORIES_BRANDS


In [ ]:
#focusing on weight of item now 


#I see the unit of weight is not constant ( I want to stick to the grams convention)
def get_weight(item):
    weight_str = item.features.get('Item Weight')
    if weight_str:
        parts = weight_str.split(' ')
        amount = float(parts[0])
        unit = parts[1].lower()
        
        if unit == "pounds":
            return amount * 453.592  #pounds to g
        elif unit == "ounces":
            return amount * 28.3495  #ounces to g
        elif unit == "grams":
            return amount  # in g
        elif unit == "milligrams":
            return amount / 1000  #mg to g
        elif unit == "kilograms":
            return amount * 1000  #kg to g
        elif unit == "hundredths" and parts[2].lower() == "pounds":
            return (amount / 100) * 453.592  #hundredths of pounds to g
        else:
            print(weight_str)
    return None

In [ ]:
#storing weights
weights = [get_weight(item) for item in train_details]
weights = [w for w in weights if w] #removing duplicates or empty 

In [ ]:
average_weight = sum(weights)/len(weights)
print(f"{average_weight:,.2f}")

#filling the empty weights with average weight 
def get_weight_with_default(item):
    weight = get_weight(item)
    return weight or average_weight

In [ ]:
#focusing on rank 
# single product has multiple ranks(in different product categories)
# so taking average of all of them 
def get_rank(item):
    rank_dict = item.features.get("Best Sellers Rank")
    if rank_dict:
        ranks = rank_dict.values()
        return sum(ranks)/len(ranks)
    return None 

ranks=[get_rank(item) for item in train_details]
ranks = [r for r in ranks if r]
average_rank = sum(ranks)/len(ranks)
print(f"{average_rank:,.2f}")

In [ ]:
#filling empty with average rank
def get_rank_with_default(item):
    rank = get_rank(item)
    return rank or average_rank

In [ ]:
def get_text_length(item):
    return len(item.test_prompt())

In [ ]:
#getting a features dictionary 
def get_features(item):
    return {
        'weight':get_weight_with_default(item) ,
        'rank':get_rank_with_default(item) ,
        'is_top_auto_parts_brand' : 1 if is_top_auto_parts_brand(item) else 0 ,
        'is_premium_electronics_brand': 1 if is_premium_electronics_brand(item) else 0 ,
        'is_top_electronics_brand': 1 if is_top_electronics_brand(item) else 0 ,
        'is_top_auto_accessories_brand': 1 if is_top_auto_accessories_brand(item) else 0 ,
        'text_length': get_text_length(item) ,
    }

In [ ]:
get_features(train_details[35])

In [ ]:
#converting to df 
def list_to_df(items):
    features = [get_features(item) for item in items]
    df = pd.DataFrame(features)
    df['price'] = [item.price for item in items]
    return df

train_details_df = list_to_df(train_details)
test_details_df = list_to_df(test_details)


### Baseline Models

#### Need to have another tester class which tests on the list of the data I have provided 

In [ ]:
class Tester_baseline:

    def __init__(self, predictor, title=None, data=test_details, size=250):
        self.predictor = predictor
        self.data = data
        self.title = title or predictor.__name__.replace("_", " ").title()
        self.size = size
        self.guesses = []
        self.truths = []
        self.errors = []
        self.sles = []
        self.colors = []

    def color_for(self, error, truth):
        if error<40 or error/truth < 0.2:
            return "green"
        elif error<80 or error/truth < 0.4:
            return "orange"
        else:
            return "red"
    
    def run_datapoint(self, i):
        datapoint = self.data[i]
        guess = self.predictor(datapoint)
        truth = datapoint.price
        error = abs(guess - truth)
        log_error = math.log(truth+1) - math.log(guess+1)
        sle = log_error ** 2
        color = self.color_for(error, truth)
        title = datapoint.title if len(datapoint.title) <= 40 else datapoint.title[:40]+"..."
        self.guesses.append(guess)
        self.truths.append(truth)
        self.errors.append(error)
        self.sles.append(sle)
        self.colors.append(color)
        print(f"{COLOR_MAP[color]}{i+1}: Guess: ${guess:,.2f} Truth: ${truth:,.2f} Error: ${error:,.2f} SLE: {sle:,.2f} Item: {title}{RESET}")

    def chart(self, title):
        max_error = max(self.errors)
        plt.figure(figsize=(12, 8))
        max_val = max(max(self.truths), max(self.guesses))
        plt.plot([0, max_val], [0, max_val], color='deepskyblue', lw=2, alpha=0.6)
        plt.scatter(self.truths, self.guesses, s=3, c=self.colors)
        plt.xlabel('Ground Truth')
        plt.ylabel('Model Estimate')
        plt.xlim(0, max_val)
        plt.ylim(0, max_val)
        plt.title(title)
        plt.show()

    def report(self):
        average_error = sum(self.errors) / self.size
        rmsle = math.sqrt(sum(self.sles) / self.size)
        hits = sum(1 for color in self.colors if color=="green")
        title = f"{self.title} Error=${average_error:,.2f} RMSLE={rmsle:,.2f} Hits={hits/self.size*100:.1f}%"
        self.chart(title)

    def run(self):
        self.error = 0
        for i in range(self.size):
            self.run_datapoint(i)
        self.report()

    @classmethod
    def test(cls, function):
        cls(function).run()

#### Linear Regression

In [ ]:
np.random.seed(42)

train_details_df.keys()

In [ ]:
feature_columns = ['weight', 'rank', 'is_top_auto_parts_brand',
       'is_premium_electronics_brand', 'is_top_electronics_brand',
       'is_top_auto_accessories_brand', 'text_length']

X_train = train_details_df[feature_columns]
Y_train = train_details_df['price']
x_test =test_details_df[feature_columns]
y_test =test_details_df['price']

model= LinearRegression()
model.fit(X_train , Y_train)

for feature , coef in zip(feature_columns , model.coef_):
    print(f"{feature}:{coef}")
print(f"Intercept :{model.intercept_}")

y_pred = model.predict(x_test)
mse = mean_squared_error(y_test , y_pred)
r2=r2_score(y_test , y_pred)

print(f"RMSE :{(mse)**0.5}")
print(f"R-squared Score :{r2}")

In [ ]:
def linear_regression_pricer(item):
    features = get_features(item)
    features_df = pd.DataFrame([features])
    return model.predict(features_df)[0]

In [ ]:
Tester_baseline.test(linear_regression_pricer)

#### BoW

In [ ]:
# 1. Data Preparation
documents = train['test_prompt'].to_list()
# test prompt so the model doesn't see the price
np.random.seed(42)
prices = train['price'].astype(float).to_numpy()
# using np array maybe much better for setting random seeds 
# also LR converts it into array so better for efficiency

# 2. Processing and Training
vectorizer = CountVectorizer(max_features=1000, stop_words='english')
X = vectorizer.fit_transform(documents)
regressor = LinearRegression()
regressor.fit(X, prices)

In [ ]:
def bow_lr_pricer(item):
    x = vectorizer.transform([item.test_prompt])
    return max(regressor.predict(x)[0],0)  #ensuring no negetive price is returned , also regressor.predict() return a numpy array

In [ ]:
Tester.test(bow_lr_pricer)

#### Word2Vec

In [ ]:
np.random.seed(42)

processed_docs = [simple_preprocess(doc) for doc in documents]

#training 
w2v_mode = Word2Vec(sentences=processed_docs, 
                        vector_size=400 , 
                        window=5,
                        min_count=2, 
                        workers=8,
                        epochs = 2 ,
                    )

                

In [ ]:
#taking average for each description of the product (not the best way-good for trial)
def document_vector(doc):
    doc_words = simple_preprocess(doc)
    word_vectors = [w2v_mode.wv[word] for word in doc_words if word in w2v_mode.wv]
    return np.mean(word_vectors , axis = 0) if word_vectors else np.zeros(w2v_mode.vector_size)

X_w2v = np.array([document_vector(doc) for doc in documents])


In [ ]:
w2v_lr_regressor = LinearRegression()
w2v_lr_regressor.fit(X_w2v,prices)

In [ ]:
def w2v_lr_pricer(item):
    doc = item.test_prompt 
    doc_vector = document_vector(doc)
    return max(0,w2v_lr_regressor.predict([doc_vector])[0])

In [ ]:
Tester.test(w2v_lr_pricer)

#### SVM

In [ ]:
from sklearn.svm import LinearSVR

In [ ]:
np.random.seed(42)
svr_regressor = LinearSVR()
svr_regressor.fit(X_w2v, prices)

In [ ]:
def svr_pricer(item):
    np.random.seed(42)
    doc = item.test_prompt
    doc_vector = document_vector(doc)
    return max(0,float(svr_regressor.predict([doc_vector])[0]))

In [ ]:
Tester.test(svr_pricer)

#### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42 , n_jobs=8)
rf_model.fit(X_w2v , prices)

In [ ]:
def random_forest_pricer(item):
    doc = item.test_prompt
    doc_vector = document_vector(doc)
    return max(0,rf_model.predict([doc_vector])[0])

In [ ]:
Tester.test(random_forest_pricer)